# ResNet with Denoised Melspectrograms

In this notebook I'll be following this for the ResNet architecture https://adventuresinmachinelearning.com/introduction-resnet-tensorflow-2/

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/rfcx-species-audio-detection/sample_submission.csv
/kaggle/input/rfcx-species-audio-detection/train_fp.csv
/kaggle/input/rfcx-species-audio-detection/train_tp.csv
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/18-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/08-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/13-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/11-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/02-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/07-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/27-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/23-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/04-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/30-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/test/01-63.tfrec
/kaggle/input/rfcx-species-audio-detection/tfrecords/

In [2]:
import os

import pandas as pd
import numpy as np

import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from tqdm import tqdm

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [3]:
BASE_DIR = '../input/rfcsaddenoisedmelspectrainv1/train_images'

train_fp = pd.read_csv('../input/rfcx-species-audio-detection/train_fp.csv')
train_fp.head()

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
1,00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
2,00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
3,003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
4,003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [4]:
NUM_SPECIES = len(set(train_fp['species_id'].values))
print('There are %s species' % (NUM_SPECIES))

There are 24 species


In [5]:
IMG_HEIGHT = 200 # 310
IMG_WIDTH = 100 # 154

images = []
species = []

for index, row in tqdm(train_fp.iterrows()):    
    # Load the image
    image_path= '%s/%s.jpg' % (BASE_DIR, str(row['recording_id']))
    image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    
    # Put the image into an array
    images = images + [image]
    
    # Put the species_id into an array
    species = species + [int(row['species_id'])] 

7781it [00:54, 143.79it/s]


The image shapes are (IMG_WIDTH, IMG_HEIGHT, 3).

In [6]:
'''
# Save Images Array
images_file = open('%s/images' % (OUTPUT_DIR), "wb")
np.save(images_file, images)
# close the file
images_file.close

# Save Species Array
species_file = open('%s/species' % (OUTPUT_DIR), "wb")
np.save(species_file, species)
# close the file
species_file.close
'''

'\n# Save Images Array\nimages_file = open(\'%s/images\' % (OUTPUT_DIR), "wb")\nnp.save(images_file, images)\n# close the file\nimages_file.close\n\n# Save Species Array\nspecies_file = open(\'%s/species\' % (OUTPUT_DIR), "wb")\nnp.save(species_file, species)\n# close the file\nspecies_file.close\n'

# Compile and Fit Model

### Define the ResNet Block

In [7]:
def res_net_block(input_data, filters, conv_size):
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, input_data])
    x = layers.Activation('relu')(x)
    return x

### Define the ResNet Model

In [8]:
inputs = keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))

x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
num_res_net_blocks = 10
for i in range(num_res_net_blocks):
    x = res_net_block(x, 64, 3)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(NUM_SPECIES, activation='softmax')(x)


res_net_model = keras.Model(inputs, outputs)

In [9]:
res_net_model.compile(optimizer=keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [10]:
res_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 98, 198, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 196, 64)  18496       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 32, 65, 64)   0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [11]:
history = res_net_model.fit(x=np.array(images, np.float32), 
                    y=np.array(species, np.float32), 
                    epochs=100)

Epoch 1/100
244/244 [==============================] - 24s 74ms/step - loss: 3.2058 - acc: 0.0685
Epoch 2/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1539 - acc: 0.0793
Epoch 3/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1543 - acc: 0.0809
Epoch 4/100
244/244 [==============================] - 18s 73ms/step - loss: 3.1523 - acc: 0.0813
Epoch 5/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1566 - acc: 0.0777
Epoch 6/100
244/244 [==============================] - 18s 73ms/step - loss: 3.1459 - acc: 0.0861
Epoch 7/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1497 - acc: 0.0800
Epoch 8/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1523 - acc: 0.0745
Epoch 9/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1515 - acc: 0.0787
Epoch 10/100
244/244 [==============================] - 18s 74ms/step - loss: 3.1554 - acc: 0.0729
Epoch 11/100
244/24